In [154]:
import pandas as pd
bay=pd.read_csv("../nb_outputs/news_20/baysian_topic_5.csv")
bay.drop(["Unnamed: 0"],axis=1,inplace=True)

plsa_5top=pd.read_csv("../plsa_outputs/news_20/5_topics.csv")
plsa_5top.drop(["Unnamed: 0"],axis=1,inplace=True)


In [155]:
import pickle
with open("../nb_outputs/news_20/missing_values_5.pkl","rb") as f:
    missed_idx=pickle.load(f) #got this from nb_gnn 
# missed_idx #drop these values from plsa  #these are the index which are not in the baysian_topic_5 dataset 

In [156]:
news_20=pd.read_csv("../datasets/20news.csv")
print(news_20.shape,bay.shape[0]+len(missed_idx))
#now using this information generate new indexes
#from the original subtract(eliminate) the missing idx and the new idx 
original_idx=list(range(news_20.shape[0]))
print(len(original_idx))
for item in missed_idx:
    original_idx.remove(item)
len(original_idx),bay.shape

(18846, 3) 18846
18846


(10743, (10743, 5))

In [157]:
# make a new column in bayes and append these indexes
bay["index"]=original_idx

# Now remove the rows which was not common to both the datasets

In [158]:
bay_idx,plsa_5top_idx=set(bay["index"]),set(plsa_5top["index"])

In [159]:
idx_to_keep=bay_idx.intersection(plsa_5top_idx)

In [160]:
len(idx_to_keep)

10736

In [169]:
cleaned_bay=bay[bay["index"].isin(idx_to_keep)]
cleaned_plsa_5=plsa_5top[plsa_5top["index"].isin(idx_to_keep)]


In [172]:
temp=cleaned_plsa_5["index"].to_list()
for i in cleaned_bay["index"].to_list():
    if i not in temp:
        print(i,"what the fuck")
    # break

# Finding match accuracy without order

In [175]:
import pandas as pd

def accuracy(n_largest,match_threshold): #instead of gntm here we have PLSA or other probabilitic models

    bay=cleaned_bay
    gnt=cleaned_plsa_5 #changing the gntm to plsa
    print("read sucessfully")
    # bay.drop(["Unnamed: 0"],axis=1,inplace=True) 
    # gnt.drop(["Unnamed: 0"],axis=1,inplace=True) 
    bay,gnt=bay.transpose(),gnt.transpose()
    bay_top_3,gnt_top_3=[],[]

    for col in range(bay.shape[1]):
        try:
            bay_top_3.append(bay.nlargest(n_largest,col).index) 
        except:
            pass

    for col in range(gnt.shape[1]):
        try:
            gnt_top_3.append(gnt.nlargest(n_largest,col).index)
        except:
            pass

    bay_col_names=[]
    gnt_col_names=[]
    for i in range(1,n_largest+1):
        bay_col_names.append(f'bay_top{str(i)}')
        gnt_col_names.append(f'gnt_top{str(i)}')
        
    bay_top3_df=pd.DataFrame(bay_top_3,columns=bay_col_names)
    gnt_top3_df=pd.DataFrame(gnt_top_3,columns=gnt_col_names)
    final_matches=[]
    for gnt_out,bay_out in zip(gnt_top_3,bay_top_3):
        matches=[]
        for out in gnt_out:
            if out in bay_out:
                matches.append(True)
            else:
                matches.append(False)
        percentage_of_acc=(matches.count(True)/len(matches))*100
        if percentage_of_acc >= match_threshold: 
            final_matches.append(True)
        else:
            final_matches.append(False)
    result=pd.concat([bay_top3_df,gnt_top3_df,pd.DataFrame(final_matches,columns=["match"])],axis=1)
    # decision=str(input("Do you want to save the o/p(yes/no)?"))
    decision="yes"
    if decision=="Yes" or decision=="yes" or decision=="YES":
        file_name=str(input("enter file name :"))
        result.to_csv(f"{file_name}.csv")
    
    try:
        result["match"].value_counts()[True]
    except:
        print('accuracy : 0 % ')
    try:
        result["match"].value_counts()[False]
    except:
        print('accuracy : 100%')

    try:
        accuracy=result["match"].value_counts()[True]/(result["match"].value_counts()[True]+result["match"].value_counts()[False])
        print("The accuracy is :",accuracy)
    except:
        print("result already shown")



topics=str(input("enter no. of topics :"))
n_largest=int(input("enter n_largest : "))

thres=[60,80,100]

for threshold in thres:
    print("topics:",topics)
    print("n largest :",n_largest)
    print("Threshold is :",threshold)
    accuracy(n_largest,threshold)

topics: 5
n largest : 3
Threshold is : 60
read sucessfully
accuracy : 0 % 
result already shown
topics: 5
n largest : 3
Threshold is : 80
read sucessfully
accuracy : 0 % 
result already shown
topics: 5
n largest : 3
Threshold is : 100
read sucessfully


PermissionError: [Errno 13] Permission denied: 'thres_80.csv'